In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('../Datasets/processed_data.csv')
df.info

<bound method DataFrame.info of        campaign_item_id  no_of_days  ext_service_id ext_service_name  \
0                  2733    0.059322             128     Facebook Ads   
1                  2733    0.067797              16            DV360   
2                  2733    0.076271             128     Facebook Ads   
3                  2733    0.084746             128     Facebook Ads   
4                  2733    0.093220               4       Google Ads   
...                 ...         ...             ...              ...   
65744              3960    0.016949             128     Facebook Ads   
65745              3960    0.025424             128     Facebook Ads   
65746              3960    0.033898             128     Facebook Ads   
65747              3960    0.042373             128     Facebook Ads   
65748              3960    0.050847              16            DV360   

       creative_id                               search_tags  template_id  \
0             1000        

In [3]:
import numpy as np
import pandas as pd

# Step 1: Load GloVe embeddings
glove_path = "../GloVe/glove.6B.50d.txt"  # Update with the correct path

def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]  # First value is the word
            vector = np.array(values[1:], dtype=np.float32)  # Rest are vector components
            embeddings[word] = vector
    return embeddings

glove_embeddings = load_glove_embeddings(glove_path)


In [4]:
# Step 2: Function to get phrase-level embedding
def get_phrase_embedding(phrase):
    words = phrase.split()  # Split phrase into words
    vectors = [glove_embeddings[word] for word in words if word in glove_embeddings]  
    if vectors:
        return np.mean(vectors, axis=0)  # Average word vectors to form phrase vector
    else:
        return np.zeros(50)  # Return zero vector if no word is found in GloVe


In [5]:
# Step 3: Apply embeddings to the DataFrame
df['keyword_embedding'] = df['keywords'].apply(get_phrase_embedding)

In [6]:
# Step 3: Apply embeddings to the DataFrame
df['advertiser_embedding'] = df['advertiser_name'].apply(get_phrase_embedding)

In [7]:
import pandas as pd

# Convert keyword embeddings to a DataFrame
keyword_embeddings_df = pd.DataFrame(df['keyword_embedding'].tolist(), 
                                     columns=[f'keyword_emb_{i}' for i in range(50)])
keyword_embeddings_df.insert(0, "keyword", df['keywords'])  # Add keyword column

# Save keyword embeddings for later use
keyword_embeddings_df.to_csv("../Datasets/keyword_embeddings.csv", index=False)

print("✅ Keyword embeddings saved to 'keyword_embeddings.csv'.")

# Merge embeddings into the original DataFrame
df = pd.concat([df.drop(columns=['keywords', 'keyword_embedding']), keyword_embeddings_df.drop(columns=['keyword'])], axis=1)

✅ Keyword embeddings saved to 'keyword_embeddings.csv'.


In [8]:
import pandas as pd
import numpy as np

# Assuming df is the original DataFrame containing a column 'advertiser_name' with advertiser names

# Function to get GloVe embedding for a given word
def get_glove_embedding(word):
    if isinstance(word, str):  # Ensure the word is a string before splitting
        words = word.split()  # Handle multi-word names
        vectors = [glove_embeddings[w.lower()] for w in words if w.lower() in glove_embeddings]
        if vectors:
            return np.mean(vectors, axis=0)  # Average vectors for multi-word names
    return np.zeros(50)  # Default zero vector for non-strings or unknown words

# Step 1: Handle non-string values in the 'advertiser_name' column (replace NaN or non-strings with empty strings)
df['advertiser_name'] = df['advertiser_name'].fillna('').astype(str)

# Step 2: Apply the embedding function to each 'advertiser_name' in the DataFrame
df['advertiser_embedding'] = df['advertiser_name'].apply(get_glove_embedding)

# Step 3: Convert the embeddings into separate columns
advertiser_embeddings_df = pd.DataFrame(df['advertiser_embedding'].tolist(), 
                                        columns=[f'advertiser_emb_{i}' for i in range(50)])

# Step 4: Include only the 'advertiser_name' and embedding columns
advertiser_embeddings_df.insert(0, "advertiser_name", df['advertiser_name'])

# Step 5: Save the embeddings to a separate CSV file (only 'advertiser_name' and embeddings)
advertiser_embeddings_df.to_csv("../Datasets/advertiser_embeddings.csv", index=False)

print("✅ Advertiser embeddings for all records saved to 'advertiser_embeddings.csv'.")


✅ Advertiser embeddings for all records saved to 'advertiser_embeddings.csv'.


In [9]:
# Function to get phrase-level embedding for search tags
def get_search_tag_embedding(tag):
    if isinstance(tag, str):  # Ensure the tag is a string before splitting
        words = tag.split()  # Split tag into words
        vectors = [glove_embeddings[word] for word in words if word in glove_embeddings]  
        if vectors:
            return np.mean(vectors, axis=0)  # Average word vectors to form phrase vector
    return np.zeros(50)  # Return zero vector if no word is found or tag is not a string

# Apply embeddings to the search_tags column
df['search_tag_embedding'] = df['search_tags'].apply(get_search_tag_embedding)

# Convert search tag embeddings to a DataFrame
search_tag_embeddings_df = pd.DataFrame(df['search_tag_embedding'].tolist(), 
                                     columns=[f'search_tag_emb_{i}' for i in range(50)])
search_tag_embeddings_df.insert(0, "search_tag", df['search_tags'])  # Add search_tag column

# Save search tag embeddings for later use
search_tag_embeddings_df.to_csv("../Datasets/search_tag_embeddings.csv", index=False)
print("✅ Search tag embeddings saved to 'search_tag_embeddings.csv'.")

# Clean up the main DataFrame without adding the embeddings
df = df.drop(columns=['search_tags', 'search_tag_embedding'])


✅ Search tag embeddings saved to 'search_tag_embeddings.csv'.
